Goal: Geocode hospital addresses to get lat/lon values. Export a combined file for use in further analysis/mapping.

First, we will take a closer look at what the adresses look like and do some geocoding-specific data cleaning. 

In [1]:
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
from tqdm import tqdm
import time
import os
import requests


# Keeping pandas from truncating long strings
pd.set_option('display.max_colwidth', None)

/Users/rattnern/.local/share/virtualenvs/scripts-uQiJCpmS/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Load data

In [2]:
file_path = '../data/yearly_hospital_lists/processed/combined_df.csv'
combined_df = pd.read_csv(file_path)

In [3]:
zip_centroids = pd.read_csv('../data/zip_code_centroids/ZIP_Code_Population_Weighted_Centroids_-8037460774014549482.csv', dtype={"STD_ZIP5": str})

### Cleaning

#### Trailing commas

In [4]:
# Remove trailing commas and leading/trailing whitespace from the address field
combined_df['ADDRESS'] = combined_df['ADDRESS'].str.strip().str.rstrip(',')

#### Things in parentheses
This is floor or building info that we can do away with

In [5]:
combined_df['ADDRESS'] = combined_df['ADDRESS'].str.replace(r'\([^)]*\)', '', regex=True).str.strip()

#### PO boxes
There are about 275 rows that contain the word "box", which will be problematic for geocoding. We will clean this data with this approach:
- If the address starts with what we deem to be a valid street address,s strip the PO Box info and just use the address to geocode.
- If the address appears to be a PO Box only with no street address, replace with a fallback of zip code centroids (from a HUD dataset)

The patch_final_box_cases function handles a few edge cases manually.

In [6]:
# --- Fallback for PO Box-only entries ---
def fallback_to_zip_centroid(address):
    zip_match = re.search(r'\b\d{5}\b', address)
    return f"ZIP_CENTER_{zip_match.group()}" if zip_match else "ZIP_CENTER_UNKNOWN"

# --- Final cleaning logic ---
# def clean_extended_box_cases(address):
#     addr = address.strip()
#     addr_upper = addr.upper()

#     # Fallback cases — treat as non-mappable
#     if re.match(r'^\s*(P\.?\s*O\.?|POST\s+OFFICE)\s+BOX', addr_upper):
#         return None
#     if re.match(r'^\s*BOX\s+[A-Z0-9]+', addr_upper):
#         return None
#     if re.search(r'\b(CALLER\s+)?BOX\s+[A-Z0-9]{1,6}', addr_upper):
#         return None

#     # Remove dangling "P O"
#     if re.search(r'\bP\s*O\b', addr_upper):
#         addr = re.sub(r',?\s*\(?\bP\s*O\b\)?', '', addr, flags=re.IGNORECASE)

#     # Remove numeric BOX forms: BOX 123, / BOX 456, BOX#789
#     addr = re.sub(r'[,/]*\s*BOX\s*#?\s*\d+\b', '', addr, flags=re.IGNORECASE)

#     return addr.strip().strip(',')

def clean_extended_box_cases(address):
    addr = address.strip()
    addr_upper = addr.upper()

    # === 1. Handle PO Box and common patterns ===
    if re.match(r'^\s*(P\.?\s*O\.?|POST\s+OFFICE)\s+BOX', addr_upper):
        return None
    if re.match(r'^\s*BOX\s+[A-Z0-9]+', addr_upper):
        return None
    if re.search(r'\b(CALLER\s+)?BOX\s+[A-Z0-9]{1,6}', addr_upper):
        return None

    # Remove dangling "P O"
    addr = re.sub(r',?\s*\(?\bP\s*O\b\)?', '', addr, flags=re.IGNORECASE)

    # Remove BOX #### or BOX A
    addr = re.sub(r'[,/]*\s*BOX\s*#?\s*\w+\b', '', addr, flags=re.IGNORECASE)

    # Normalize "U S HIGHWAY" → "US Hwy"
    # addr = re.sub(r'\bU\s*S\s*HIGHWAY\b', 'US Hwy', addr, flags=re.IGNORECASE)

    # Remove STE ###, SUITE ###, FLOOR ###, BLDG ###
    addr = re.sub(r'\b(STE|STE\.|SUITE|FLOOR|BLDG)[\s#]*\d*[A-Z]*\b', '', addr, flags=re.IGNORECASE)

    # === 🔢 Convert spelled-out numbers at start of address ===
    number_map = {
        'ONE': '1', 'TWO': '2', 'THREE': '3', 'FOUR': '4', 'FIVE': '5',
        'SIX': '6', 'SEVEN': '7', 'EIGHT': '8', 'NINE': '9', 'TEN': '10'
    }
    for word, digit in number_map.items():
        addr = re.sub(rf'^\b{word}\b(?=\s)', digit, addr, flags=re.IGNORECASE)

    # Final cleanup
    addr = re.sub(r'\s{2,}', ' ', addr).strip()
    addr = re.sub(r',\s*,', ',', addr).strip(',')
    return addr

# --- Manual patch for known cases that slipped through ---
def patch_final_box_cases(address):
    address = address.strip()
    if address.upper().startswith("420 34TH ST BOX"):
        return "420 34TH ST"
    elif "GIBSON BOULEVARD" in address.upper():
        return "5400 GIBSON BOULEVARD SE, 4TH FLOOR"
    elif "225 E CHICAGO" in address.upper():
        return "225 E CHICAGO"
    else:
        return address

In [7]:
# Initial clean
combined_df['cleaned_address'] = combined_df['ADDRESS'].apply(clean_extended_box_cases)

# ZIP centroid fallback for unmappable rows
combined_df['cleaned_address'] = combined_df.apply(
    lambda row: fallback_to_zip_centroid(row['ADDRESS']) if pd.isna(row['cleaned_address']) else row['cleaned_address'],
    axis=1
)

# Manual patching
combined_df['cleaned_address'] = combined_df['cleaned_address'].apply(patch_final_box_cases)

In [8]:
combined_df[combined_df['cleaned_address'].str.contains('ZIP')]

,ID,NAME,ADDRESS,CITY,STATE,ZIP,FIPS,RUCA,RURAL_STATUS,TOTAL_BEDS,ACUTE_BEDS,YEAR,HOSPITAL_TYPE,TYPE,TOTAL_BEDS.1,cleaned_address
35,10073,CLAY COUNTY HOSPITAL,83825 HIGHWAY 9 P O BOX 1270,ASHLAND,AL,36251,1027,2.0,Rural,53.0,46.0,2023,Acute,NaN,NaN,ZIP_CENTER_83825
83,11304,OCHSNER CHOCTAW GENERAL,"401 VANITY FAIR LANE, PO BOX 618",BUTLER,AL,36904,1023,10.0,Rural,25.0,25.0,2023,Acute,NaN,NaN,ZIP_CENTER_UNKNOWN
91,20018,YUKON KUSKOKWIM DELTA REG HOSPITAL,PO BOX 287,BETHEL,AK,99559,2050,NaN,Rural,50.0,34.0,2023,Acute,NaN,NaN,ZIP_CENTER_UNKNOWN
94,21301,PROVIDENCE VALDEZ MEDICAL CENTER,PO BOX 550,VALDEZ,AK,99686,2063,NaN,Rural,11.0,11.0,2023,Acute,NaN,NaN,ZIP_CENTER_UNKNOWN
95,21302,PROVIDENCE SEWARD MEDICAL CENTER,"417 FIRST AVENUE, PO BOX 365",SEWARD,AK,99664,2122,10.0,Rural,6.0,6.0,2023,Acute,NaN,NaN,ZIP_CENTER_UNKNOWN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54157,494022,POPLAR SPRINGS HOSPITAL,350 POPLAR DRIVE PO BOX 3060,PETERSBURG,VA,23805,51730.0,1.0,Urban,130.0,95.0,2021,Specialty,PSYCH,NaN,ZIP_CENTER_UNKNOWN
54162,501992,SUNRISE HAVEN,PO BOX 6057 24423 100TH AVENUE SOUTH,KENT,WA,98064,53033.0,1.0,Urban,8.0,4.0,2021,Specialty,RELIGIOUS NON-MED,NaN,ZIP_CENTER_24423
54166,503300,SEATTLE CHILDREN'S HOSPITAL,"4800 SAND POINT WAY NE, PO BOX C-5371",SEATTLE,WA,98105,53033.0,1.0,Urban,250.0,343.0,2021,Specialty,CHILD,NaN,ZIP_CENTER_UNKNOWN
54167,503301,MARY BRIDGE CHILDREN'S HOSPITAL,317 MARTIN LUTHER KING JR W BOX 5299,TACOMA,WA,98415,53053.0,1.0,Urban,68.0,NaN,2021,Specialty,CHILD,NaN,ZIP_CENTER_UNKNOWN


### Build full address column

In [9]:
# Build full address 
combined_df['full_address'] = (
    combined_df['cleaned_address'] + ', ' +
    combined_df['CITY'] + ', ' +
    combined_df['STATE'] + ' ' +
    combined_df['ZIP'].astype(str)
)

combined_df.shape

(54262, 17)

In [10]:
# Drop duplicate addresses to save time/bandwidth on geocoding
unique_addresses = combined_df[['full_address']].drop_duplicates().reset_index(drop=True)

unique_addresses.shape

(7384, 1)

### Look for remaining potential issues before geocoding

In [11]:
# See if we have any missing values that we need for geocoding
cols_to_check = ['NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP']
missing_rows = combined_df[combined_df[cols_to_check].isnull().any(axis=1)]
print(f"Rows with missing values: {len(missing_rows)}")

Rows with missing values: 0


In [12]:
# Null or blank
nulls = unique_addresses[unique_addresses['full_address'].isnull() | (unique_addresses['full_address'].str.strip() == '')]
print(f"Null or blank addresses: {len(nulls)}")

# Addresses that are unusually short
short = unique_addresses[unique_addresses['full_address'].str.len() < 15]
print(f"Suspiciously short addresses (<15 chars): {len(short)}")

# Addresses with repeated commas (may indicate missing fields)
messy = unique_addresses[unique_addresses['full_address'].str.contains(r',\s*,')]
print(f"Addresses with double commas: {len(messy)}")

Null or blank addresses: 0
Suspiciously short addresses (<15 chars): 0
Addresses with double commas: 0


### Geocode

In [13]:
# # Step 1: Set up output paths
# out_dir = "../data/yearly_hospital_lists/geocoded/"
# os.makedirs(out_dir, exist_ok=True)
# partial_path = out_dir + "geocoded_partial.csv"
# final_path = out_dir + "geocoded_final.csv"

# # Step 2: Filter to geocodable addresses (exclude ZIP_CENTER fallback)
# to_geocode = unique_addresses[~unique_addresses['full_address'].str.startswith("ZIP_CENTER_")].copy()
# to_geocode = to_geocode.head(150)

# # Step 3: Load previously geocoded progress if available
# try:
#     geocoded = pd.read_csv(partial_path)
#     already_geocoded = set(geocoded['full_address'])
# except FileNotFoundError:
#     geocoded = pd.DataFrame(columns=['full_address', 'latitude', 'longitude'])
#     already_geocoded = set()

# # Step 4: Set up geocoder and rate limiter
# geolocator = Nominatim(user_agent="hospital_access_geocoder")
# rate_limited_geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# # Step 5: Main geocoding loop with checkpoint saving
# results = []

# for i, row in tqdm(to_geocode.iterrows(), total=len(to_geocode), desc="Geocoding"):
#     addr = row['full_address']

#     if addr in already_geocoded:
#         continue

#     try:
#         location = rate_limited_geocode(addr)
#         lat, lon = (location.latitude, location.longitude) if location else (None, None)
#     except Exception:
#         lat, lon = None, None

#     results.append({'full_address': addr, 'latitude': lat, 'longitude': lon})

#     # Save checkpoint every 50 rows
#     if len(results) % 50 == 0:
#         temp_df = pd.DataFrame(results)
#         combined = pd.concat([geocoded, temp_df], ignore_index=True)
#         combined.to_csv(partial_path, index=False)

# # Step 6: Final save
# final_geocoded = pd.concat([geocoded, pd.DataFrame(results)], ignore_index=True)
# final_geocoded.to_csv(final_path, index=False)

# # Summary
# print(f"✅ Geocoding complete. {len(final_geocoded)} total rows saved to:\n{final_path}")

In [14]:
def full_geocode_pipeline_with_checkpoints(to_geocode_df, zip_center_df, zip_centroids, save_path):
    import pandas as pd
    import requests
    from geopy.geocoders import Nominatim
    from geopy.extra.rate_limiter import RateLimiter
    from tqdm import tqdm

    # Setup
    geolocator = Nominatim(user_agent="hospital_access_pipeline")
    rate_limited_geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

    # === Safe Nominatim wrapper ===
    def safe_nominatim_geocode(x):
        loc = rate_limited_geocode(x)
        if loc:
            return pd.Series([loc.latitude, loc.longitude])
        else:
            return pd.Series([None, None])

    # === Census API ===
    def census_geocode(address):
        url = "https://geocoding.geo.census.gov/geocoder/locations/onelineaddress"
        params = {"address": address, "benchmark": "Public_AR_Current", "format": "json"}
        response = requests.get(url, params=params)
        try:
            match = response.json()['result']['addressMatches'][0]
            return match['coordinates']['y'], match['coordinates']['x']
        except (IndexError, KeyError):
            return None, None

    # Add lat/lon columns if not present
    for col in ['lat_census', 'lon_census', 'lat_nominatim', 'lon_nominatim']:
        if col not in to_geocode_df.columns:
            to_geocode_df[col] = None

    # Split into chunks
    chunks = [to_geocode_df.iloc[i:i+50].copy() for i in range(0, len(to_geocode_df), 50)]
    geocoded_chunks = []

    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i+1} of {len(chunks)}")

        # === Census step ===
        needs_census = chunk[chunk['lat_census'].isna()].copy()
        tqdm.pandas(desc="Census geocoding")
        needs_census[['lat_census', 'lon_census']] = needs_census['full_address'].progress_apply(
            lambda x: pd.Series(census_geocode(x))
        )
        chunk.update(needs_census)

        # === Nominatim step ===
        needs_nominatim = chunk[
            chunk['lat_census'].isna() & chunk['lat_nominatim'].isna()
        ].copy()
        tqdm.pandas(desc="Nominatim geocoding")
        needs_nominatim[['lat_nominatim', 'lon_nominatim']] = needs_nominatim['full_address'].progress_apply(
            safe_nominatim_geocode
        )
        chunk.update(needs_nominatim)

        # Save progress after each chunk
        geocoded_chunks.append(chunk)
        pd.concat(geocoded_chunks).to_csv(save_path, index=False)

        print(f"✅ Chunk {i+1}/{len(chunks)} complete — saved {len(pd.concat(geocoded_chunks))} rows")

    # Combine all geocoded chunks
    full_geocoded = pd.concat(geocoded_chunks, ignore_index=True)

    # ZIP fallback
    full_geocoded['zip5'] = full_geocoded['full_address'].str.extract(r'(\d{5})$')
    full_geocoded = full_geocoded.merge(zip_centroids[['STD_ZIP5', 'LATITUDE', 'LONGITUDE']],
                                        left_on='zip5', right_on='STD_ZIP5', how='left')

    full_geocoded['final_latitude'] = full_geocoded['lat_census'].combine_first(
        full_geocoded['lat_nominatim']
    ).combine_first(full_geocoded['LATITUDE'])

    full_geocoded['final_longitude'] = full_geocoded['lon_census'].combine_first(
        full_geocoded['lon_nominatim']
    ).combine_first(full_geocoded['LONGITUDE'])

    # Fallback flags + source
    full_geocoded['used_zip_fallback'] = (
        full_geocoded['final_latitude'].eq(full_geocoded['LATITUDE']) &
        full_geocoded['lat_census'].isna() &
        full_geocoded['lat_nominatim'].isna()
    )

    def determine_source(row):
        if pd.notna(row['lat_census']):
            return "Census"
        elif pd.notna(row['lat_nominatim']):
            return "Nominatim"
        elif row['used_zip_fallback']:
            return "ZIP"
        else:
            return "None"

    full_geocoded['geocode_source'] = full_geocoded.apply(determine_source, axis=1)

    # ZIP_CENTER rows
    zip_center_df['zip5'] = zip_center_df['full_address'].str.extract(r'ZIP_CENTER_(\d{5})')
    zip_center_df = zip_center_df.merge(zip_centroids[['STD_ZIP5', 'LATITUDE', 'LONGITUDE']],
                                        left_on='zip5', right_on='STD_ZIP5', how='left')
    zip_center_df['final_latitude'] = zip_center_df['LATITUDE']
    zip_center_df['final_longitude'] = zip_center_df['LONGITUDE']
    zip_center_df['used_zip_fallback'] = True
    zip_center_df['geocode_source'] = "ZIP"

    # Combine everything
    final_df = pd.concat([full_geocoded, zip_center_df], ignore_index=True)
    return final_df


In [ ]:
to_geocode = unique_addresses[~unique_addresses['full_address'].str.startswith("ZIP_CENTER_")].copy()
zip_center_rows = unique_addresses[unique_addresses['full_address'].str.startswith("ZIP_CENTER_")].copy()

result_df = full_geocode_pipeline_with_checkpoints(
    to_geocode_df=to_geocode,
    zip_center_df=zip_center_rows,
    zip_centroids=zip_centroids,
    save_path="../data/yearly_hospital_lists/geocoded/in_progress_geocoding.csv"
)


Processing chunk 1 of 144


Nominatim geocoding: 100%|██████████████████████| 12/12 [00:14<00:00,  1.20s/it]


✅ Chunk 1/144 complete — saved 50 rows
Processing chunk 2 of 144


Nominatim geocoding: 100%|██████████████████████| 13/13 [00:12<00:00,  1.05it/s]


✅ Chunk 2/144 complete — saved 100 rows
Processing chunk 3 of 144


...

Nominatim geocoding: 100%|████████████████████████| 8/8 [00:07<00:00,  1.08it/s]


✅ Chunk 91/144 complete — saved 4550 rows
Processing chunk 92 of 144


Nominatim geocoding: 100%|████████████████████████| 6/6 [00:07<00:00,  1.29s/it]


✅ Chunk 92/144 complete — saved 4600 rows
Processing chunk 93 of 144


Census geocoding:   8%|██                        | 4/50 [00:00<00:08,  5.63it/s]

In [ ]:
result_df.to_csv("../data/yearly_hospital_lists/geocoded/geocoded.csv", index=False)

In [ ]:
result_df[result_200['geocode_source']=='None']